# Mots

In [1]:
!pip install pandas
!pip install spacy
!pip install Counter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 65.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Counter: filename=Counter-1.0.0-py3-none-any.whl size=5393 sha256=8e1916658dc5e30ec0bc6dd48d7d8dace4c9c2c44d4c62e74ef2ec363831ff52
  Stored in directory: /home/onyxia/.cache/pip/wheels/16/ff/7a/6e8bf2fdadb47c50a03bb4b9a59bd2b1da1b876faf8e3815d9
Successfully built Counter


In [ ]:
import pandas as pd
import spacy
from collections import Counter

# Charger les données
df = pd.read_excel('tweets_fusionnes.xlsx')

# Ajouter une colonne YearMonth si elle n'existe pas déjà
if 'YearMonth' not in df.columns:
    df['YearMonth'] = df['ConvertedDate'].dt.strftime('%Y-%m')

# Charger le modèle SpaCy
nlp = spacy.load("en_core_web_sm")

# Stopwords personnalisés (ajouter ici les mots que vous voulez exclure)
custom_stopwords = {"https", "twitter", "elon", "musk", "people","quit","leave","stay", "like", "amp", "com", "user", "retweet", "x"}

def clean_and_tokenize_spacy(text):
    """
    Utiliser SpaCy pour tokeniser et nettoyer le texte.
    """
    if not isinstance(text, str):
        return []
    doc = nlp(text.lower())
    tokens = [
        token.lemma_  # Récupérer le lemme (forme de base du mot)
        for token in doc
        if not token.is_stop  # Exclure les stopwords de SpaCy
        and token.is_alpha  # Exclure les caractères non alphabétiques
        and token.lemma_ not in custom_stopwords  # Exclure les mots personnalisés
    ]
    return tokens

# Appliquer le nettoyage et la tokenisation avec SpaCy
df['CleanedText'] = df['Content'].apply(clean_and_tokenize_spacy)

# Grouper par mois et compter les mots
monthly_top_words = {}

for month, group in df.groupby('YearMonth'):
    all_words = [word for text in group['CleanedText'] for word in text]
    word_counts = Counter(all_words)
    top_words = word_counts.most_common(10)
    monthly_top_words[month] = top_words

# Convertir les résultats en DataFrame
result_df = pd.DataFrame.from_dict(monthly_top_words, orient='index', columns=[f'Word_{i+1}' for i in range(10)])
result_df.reset_index(inplace=True)
result_df.rename(columns={'index': 'YearMonth'}, inplace=True)

# Sauvegarder dans un fichier Excel
result_df.to_excel('top_words_per_month_spacy_filtered.xlsx', index=False)

print("Les 10 mots les plus récurrents par mois ont été calculés et sauvegardés dans 'top_words_per_month_spacy_filtered.xlsx'.")
